Within a single batch, you must have the same number of timesteps (this is typically where you see 0-padding and masking). But between batches there is no such restriction. During inference, you can have any length.  
(https://datascience.stackexchange.com/questions/26366/training-an-rnn-with-examples-of-different-lengths-in-keras)

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

def rnn_model(num_tokens, num_units):
    # encoder
    enc_input = Input(shape = (None, num_tokens))
    enc_output, enc_state = tf.keras.layers.GRU(num_units, return_state=True)(enc_input) # input = [batch, timesteps, feature]
    
    # decoder
    dec_input = Input(shape = (None, num_tokens))
    dec_output = tf.keras.layers.GRU(num_units, return_sequences=True)(dec_input, initial_state = enc_state)
    
    # match input and ouput size
    fin_output = Dense(num_tokens, activation='softmax')(dec_output)

    sequence_autoencoder = Model([enc_input, dec_input], fin_output)
    
    encoder = Model(enc_input, enc_output)
    
    return sequence_autoencoder, encoder

C:\Users\Leonardo\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Leonardo\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Leonardo\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Leonardo\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWa

In [14]:
# create the model
num_features = 26 # number of features per sample (Mel)
num_units = 3 # GRU units in encoder and decoder
autoenc, enc = rnn_model(num_features, num_units)

In [20]:
# produce Mel features
from python_speech_features import mfcc
import scipy.io.wavfile as wav

# creating a fake trainset set with 10 samples, just repeating one sample wav file
X_train = []
for i in range (0, 10):
    (rate, sig) = wav.read("wav_example.wav")
    mfcc_feat = mfcc(sig, samplerate=rate,winlen=0.025,winstep=0.01,numcep=num_features,
                     nfilt=26,nfft=512,lowfreq=0,highfreq=None,preemph=0.97,
                     ceplifter=22,appendEnergy=True)
    X_train.append(mfcc_feat)
X_train = np.array(X_train)

# decoder input, as shifted encoder input
X_train_shifted = np.zeros(X_train.shape)
# loop in timesteps
for sample in range(0, X_train.shape[0]):
    for timestep in range(0, X_train.shape[1] - 1):
        X_train_shifted[sample, timestep + 1, :] = X_train[sample, timestep, :]

# train the model
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
autoenc.compile(optimizer = opt, loss = tf.keras.losses.MeanSquaredError(), metrics = ["accuracy"])
autoenc.fit(x = [X_train, X_train_shifted], y = X_train, epochs = 100, batch_size = 256)

C:\Users\Leonardo\anaconda3\lib\site-packages\ipykernel_launcher.py:7: WavFileWarning: Chunk (non-data) not understood, skipping it.
  import sys


Epoch 1/100
10/10 [==============================] - 5s 495ms/sample - loss: 165.6245 - acc: 0.4421
Epoch 2/100
10/10 [==============================] - 5s 486ms/sample - loss: 165.6016 - acc: 0.4421
Epoch 3/100
10/10 [==============================] - 5s 491ms/sample - loss: 165.5780 - acc: 0.4421
Epoch 4/100
10/10 [==============================] - 5s 481ms/sample - loss: 165.5553 - acc: 0.4421
Epoch 5/100
10/10 [==============================] - 5s 486ms/sample - loss: 165.5328 - acc: 0.4421
Epoch 6/100
10/10 [==============================] - 5s 485ms/sample - loss: 165.5102 - acc: 0.4421
Epoch 7/100
10/10 [==============================] - 5s 483ms/sample - loss: 165.4876 - acc: 0.4421
Epoch 8/100
10/10 [==============================] - 5s 495ms/sample - loss: 165.4648 - acc: 0.4421
Epoch 9/100
10/10 [==============================] - 5s 487ms/sample - loss: 165.4420 - acc: 0.4421
Epoch 10/100
10/10 [==============================] - 5s 487ms/sample - loss: 165.4191 - acc: 0.4421

KeyboardInterrupt: 